<a href="https://colab.research.google.com/github/Red29-cntrl/Dolosa_Elect2/blob/Lab4/Act_4_BDA_Dolosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

In [3]:
from google.colab import files

uploaded = files.upload()  # Upload the file


Saving insurance.csv to insurance.csv


In [5]:
import os

# List files in the content directory
print(os.listdir("/content/"))


['.config', 'insurance.csv', 'sample_data']


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc

# Initialize Spark session
spark = SparkSession.builder.appName("InsuranceDataProcessing").getOrCreate()

# Load CSV file into a DataFrame
file_path = "/content/insurance.csv"  # Use full path in Colab
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show DataFrame schema and preview data
df.printSchema()
df.show(5)


root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows



In [7]:
# Select relevant columns
selected_df = df.select("age", "sex", "bmi", "children", "smoker", "region", "charges")
selected_df.show(5)

# Filter: Get only non-smokers
non_smokers_df = selected_df.filter(col("smoker") == "no")
non_smokers_df.show(5)

# Clean: Drop duplicates
cleaned_df = selected_df.dropDuplicates()
cleaned_df.show(5)


+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  

In [8]:
# Register DataFrame as a SQL table
df.createOrReplaceTempView("insurance_data")

# Example SQL Query: Get top 5 most frequent regions
top_regions = spark.sql("""
    SELECT region, COUNT(*) AS count
    FROM insurance_data
    GROUP BY region
    ORDER BY count DESC
    LIMIT 5
""")
top_regions.show()

# Grouping: Average charges per region
avg_charges_per_region = df.groupBy("region").avg("charges")
avg_charges_per_region.show()


+---------+-----+
|   region|count|
+---------+-----+
|southeast|  364|
|northwest|  325|
|southwest|  325|
|northeast|  324|
+---------+-----+

+---------+------------------+
|   region|      avg(charges)|
+---------+------------------+
|northwest|12417.575373969228|
|southeast|14735.411437609895|
|northeast|  13406.3845163858|
|southwest| 12346.93737729231|
+---------+------------------+



In [10]:
# Save results to CSV
df.write.mode("overwrite").csv("/content/insurance_cleaned.csv", header=True)

# Save results to JSON
df.write.mode("overwrite").json("/content/insurance_cleaned.json")

# Save results to TXT (only works with single-column DataFrames)
df.select("region").write.mode("overwrite").text("/content/insurance_regions")

print("Data processing and export completed successfully!")


Data processing and export completed successfully!


In [13]:
from google.colab import files
import shutil

# Zip the CSV directory before downloading
shutil.make_archive("/content/insurance_cleaned_csv", 'zip', "/content/insurance_cleaned.csv")
files.download("/content/insurance_cleaned_csv.zip")

# Zip the JSON directory before downloading
shutil.make_archive("/content/insurance_cleaned_json", 'zip', "/content/insurance_cleaned.json")
files.download("/content/insurance_cleaned_json.zip")

# Zip the TXT directory before downloading
shutil.make_archive("/content/insurance_regions_txt", 'zip', "/content/insurance_regions")
files.download("/content/insurance_regions_txt.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>